# Hugging_Face_smol_agent_contract_search

Taken from Lab agent notbook: HuggingFace smolagents: Lightweight and Focused Agents

**Objective:** This notebook introduces `smolagents`, a lightweight and easy-to-understand agent framework from HuggingFace. You will learn its core philosophy and how to build a simple, tool-using agent based on the official documentation.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Philosophy:** Simplicity and clarity. `smolagents` is not designed to be a sprawling, all-encompassing framework. Instead, it provides a minimal, clean, and effective implementation of a tool-using agent. It's an excellent choice for learning the fundamentals of the ReAct (Reason+Act) loop or for projects that need a simple, focused agent without a lot of overhead.

## 1. Setup

We will install `smolagents` and its dependencies. As per the official documentation, we'll include the `[litellm]` extra, which is required to use OpenAI models like GPT-4o.

In [1]:
import os
import json
from dotenv import load_dotenv
import sys

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    print("ERROR: OPENAI_API_KEY not found. Please check your .env file.")

## 2. Foundational Agent with a Custom Tool

The primary use case for `smolagents` is creating agents that can use tools. To ensure compatibility, we must define our custom tools as classes that inherit from the library's `Tool` base class and implement a `forward` method for the execution logic.

The process involves:
1.  Defining a custom tool class that inherits from `smolagents.tools.Tool`.
2.  Initializing the `CodeAgent` with a `LiteLLMModel` wrapper and an instance of our custom tool class.
3.  Running the agent with a prompt.

In [1]:
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool
from smolagents.tools import Tool
from typing import ClassVar

# 1. Define a custom tool class inheriting from smolagents' Tool
class ScrapeContractData(Tool):
    """Tool to webscrape data on awarded government contracts."""
    name: ClassVar[str] = "scrape_contract_data"
    description: ClassVar[str] = "Scrapes data on awarded government contracts."
    inputs: ClassVar[dict] = {
        "contract_id": {
            "type": "string",
            "description": "The ID of the government contract."
        }
    }
    output_type: ClassVar[str] = "string"

# determines the winner based on simple rules
    def forward(self, contract_id: str) -> str:
        # Placeholder implementation
        return f"Scraped data for contract ID {contract_id}"

# 2. Initialize the Model using the LiteLLMModel wrapper for OpenAI models
model = LiteLLMModel(model_id="gpt-4o")

# 3. Initialize the Agent with an instance of our custom tool
smol_agent = CodeAgent(
    model=model, 
    tools=[DuckDuckGoSearchTool()]
)

seed_data_format = """
[
  {
    "ContractName": "Example Contract 1",
    "CompanyName": "Acme Corp",
    "TotalContractValue": "$1,000,000",
    "DateAwarded": "2024-05-10",
    "Location": "Washington, DC",
    "LengthOfContract": "3 years",
    "NAICS": "541330",
    "PSC": "R499"
  },
]
"""


# 4. Run the agent
print("--- Running smol-agent ---")
# The .run() method will print the verbose thought process by default.
response = smol_agent.run(
    f"""For the top 10 U.S. government awarded contracts, over the past two years find the following information:
                          
    - Contract Name, 
    - CompanyName, 
    - TotalContractValue, 
    - Date Awarded, 
    - Location, 
    - Length of Contract and NAICS or PSC categories if available.

    If you cannot find all the information for a contract, include as much as you can and use "N/A" for missing fields.

    Extract the data and format it as a JSON array with the following format:
    {seed_data_format}

    Output of final JSON array must be wrapped in <code> ... </code> tags.
    
    """)

print(f"\n--- Final Answer ---")


# Save the response and cleaned code to artifacts
if response:    
    print(response)
    save_artifact(response, "app/HuggingFace_smol_agent_contract_search_raw.md", overwrite=True)
    print("Saved raw response to HuggingFace_smol_agent_contract_search_raw.md")
    cleaned_code = clean_llm_output(response, language='json')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/contract_search_cleaned.json", overwrite=True)
    print("Saved final answer to contract_search_cleaned.json")
else:
    print("Unable to save artifacts. Response is empty.")





--- Running smol-agent ---


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For the top 10 U.S. government awarded contracts, over the past two years find the following information:       │
│                                                                                                                 │
│     - Contract Name,                                                                                            │
│     - CompanyName,                                                                                              │
│     - TotalContractValue,                                                                                       │
│     - Date Awarded,                                                                                             │
│     - Location,                                                                                                 │
│     - Length of Contract and NAICS or PSC categories if available.                                              │
│                                                                                                                 │
│     If you cannot find all the information for a contract, include as much as you can and use "N/A" for missing │
│ fields.                                                                                                         │
│                                                                                                                 │
│     Extract the data and format it as a JSON array with the following format:                                   │
│                                                                                                                 │
│ [                                                                                                               │
│   {                                                                                                             │
│     "ContractName": "Example Contract 1",                                                                       │
│     "CompanyName": "Acme Corp",                                                                                 │
│     "TotalContractValue": "$1,000,000",                                                                         │
│     "DateAwarded": "2024-05-10",                                                                                │
│     "Location": "Washington, DC",                                                                               │
│     "LengthOfContract": "3 years",                                                                              │
│     "NAICS": "541330",                                                                                          │
│     "PSC": "R499"                                                                                               │
│   },                                                                                                            │
│ \]                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
│     Output of final JSON array must be wrapped in <code> ... </code> tags.                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            To gather the required information on the top 10 U.S. government awarded contracts over the past two 
years, we will need to search for this specific data from relevant sources such as government databases or websites
that track government contracts. This task involves multiple steps: 
1. Searching for webpages with information on top government contracts.
2. Extracting the relevant data fields from those sources.
3. Formatting the data into the desired JSON format.

Let's start by searching for the top 10 government awarded contracts in the U.S. over the past two years.</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 3.65 seconds| Input tokens: 2,155 | Output tokens: 110]

...

[Step 20: Duration 3.13 seconds| Input tokens: 209,527 | Output tokens: 4,182]

Reached max steps.

[Step 21: Duration 8.53 seconds| Input tokens: 225,574 | Output tokens: 4,468]


--- Final Answer ---
Based on the information available from USAspending, I have compiled a list of some of the major U.S. government contracts awarded over the past two years. Here is the JSON representation of the data as requested:

<code>
[
  {
    "ContractName": "Contract with Ranger Land Systems, Inc.",
    "CompanyName": "Ranger Land Systems, Inc.",
    "TotalContractValue": "$118,680,672",
    "DateAwarded": "2023-09-18",
    "Location": "N/A",
    "LengthOfContract": "N/A",
    "NAICS": "N/A",
    "PSC": "N/A"
  },
  {
    "ContractName": "Contract with AT&T Enterprises, LLC",
    "CompanyName": "AT&T Enterprises, LLC",
    "TotalContractValue": "$23,589,233",
    "DateAwarded": "2023-02-06",
    "Location": "N/A",
    "LengthOfContract": "N/A",
    "NAICS": "N/A",
    "PSC": "N/A"
  }
  // Further entries would be included similarly
]
</code>

Note: Some data fields could not be filled due to limitations in available data sources. If further details are needed, each contrac

NameError: name 'save_artifact' is not defined

## 3. Advanced Capability: Iterative Refinement (Code Generation)

`smolagents` can also be used for iterative tasks like code generation and refinement. We can simulate a developer providing feedback to the agent to improve its code.

In [ ]:
# For this example, we'll create an agent with no tools, focusing on its reasoning.
model = LiteLLMModel(model_id="gpt-4o")

code_gen_agent = CodeAgent(
    model=model,
    tools=[]
)

print("-review prompt-")
# The system prompt / persona is now part of the task given to the .run() method
prompt = code_gen_agent.run(f"You are an expert Anylist. over time anylize how sports team will do and determine what is a likely "
                                  f"case for their future performance. " \
                                  f"Consider factors like player statistics, team dynamics, and historical performance."
                                  f"start the discution using data from the Bangles and the Lions. {response}")


print(f"\n--- Original Prompt ---\n{prompt}")

print("\n--- Debate stance ---")
# We can continue the conversation by passing the previous response as context.
refined_code = code_gen_agent.run(
    f"Debate the position of which team is better. Be sure to be biased on the Bangles and use data from the previous analysis."
)
print(f"""\n--- set stance
       ---\n{refined_code}""")

## Lab Conclusion

This lab has provided a concise introduction to `smolagents`. You've learned how to create a simple, tool-using agent and how to use the agent for iterative refinement tasks.

**Key Takeaways:**
- `smolagents` is a great choice for projects that require a simple, lightweight, and easy-to-understand agent.
- It provides a clear implementation of the ReAct loop for tool use.
- Creating custom tools by inheriting from the library's `Tool` class and implementing a `forward` method with a matching signature is the correct way to ensure compatibility.